# Import packages

In [9]:
## Generic packages that are on your operating system
import glob
import os
import time

## Additional required packages
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

## Important scikit-learn packages for fitting logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import SplineTransformer, PolynomialFeatures
from sklearn.calibration import calibration_curve


## Required definitions and constants

In [28]:
files = glob.glob('datasets/IMERG*_Nairobi.zarr')
ds_IMERG = xr.open_mfdataset(files,engine="zarr")

latitude_IMERG = ds_IMERG.latitude.values
longitude_IMERG = ds_IMERG.longitude.values

box_left = longitude_IMERG-0.05
box_right = longitude_IMERG+0.05
box_down = latitude_IMERG-0.05
box_up = latitude_IMERG+0.05


def bounding_box(region,points,min_x=box_left,max_x=box_right,min_y=box_down,max_y=box_up):

    """
    points: latitude, longitude
    x: longitude
    y: latitude
    """

    section_x = []
    section_y = []
    
    for x_min, x_max in zip(box_left,box_right):
        if region == 'J.K.I.A.':
            section_x.append(np.logical_and(points[1]<=x_max+0.01,points[1]>=x_min-0.01))
        else:
            section_x.append(np.logical_and(points[1]<=x_max,points[1]>=x_min))
            
    for y_min, y_max in zip(box_down,box_up):    
        section_y.append(np.logical_and(points[0]<=y_max,points[0]>=y_min))

    idx = np.squeeze(np.argwhere(np.logical_and(np.array(section_x),np.array(section_y))))

    return idx

# Load in data

In [6]:
files = glob.glob('datasets/cGAN_50*_Nairobi.zarr')
ds_cGAN_50 = xr.open_mfdataset(files,engine="zarr")

files = glob.glob('datasets/cGAN_2*_Nairobi.zarr')
ds_cGAN = xr.open_mfdataset(files,engine="zarr")

meta = pd.read_excel('SEWAA-data.xlsx',sheet_name='Gridded',index_col=0,nrows=2)
data = pd.read_excel('SEWAA-data.xlsx',sheet_name='Gridded',index_col=0,skiprows=[1,2])

ds = data.to_xarray().rename({'ID':'time'})

# Not sure if this is the best way but it works
ds['time'] = np.array([t[:4]+'-'+t[4:6]+'-'+t[-2:] for t in ds['time'].values.astype(str)],dtype='datetime64[ns]')


In [41]:
train_years = [2018,2019,2020]

train_data_x = ds_cGAN.sel(time=ds_cGAN.time.dt.year.isin(train_years))

train_valid_times = np.intersect1d(train_data_x.time.values + np.timedelta64(1,'D'),
                                   ds.time.values)
train_data_x = ds_cGAN.sel({'time':train_valid_times - np.timedelta64(1,'D')})

train_data_y = ds.sel({'time':train_valid_times})

train_data_y

<xarray.Dataset> Size: 39kB
Dimensions:    (time: 705)
Coordinates:
  * time       (time) datetime64[ns] 6kB 2018-01-02 2018-01-03 ... 2020-12-30
Data variables:
    KABETE     (time) float64 6kB 0.385 13.36 4.398 0.0 ... 0.0 0.0 0.0 4.174
    EASTLEIGH  (time) float64 6kB 6.645 18.03 5.776 0.0 ... 0.0 0.0 0.0 19.12
    J.K.I.A.   (time) float64 6kB 3.367 20.05 1.417 1.094 ... 0.0 0.0 17.44
    DAGORETTI  (time) float64 6kB 3.038 6.752 5.673 0.0 ... 0.0 0.0 0.0 3.892
    NGONG      (time) float64 6kB 4.642 10.11 7.531 1.418 ... 0.0 0.0 0.0 3.567
    WILSON     (time) float64 6kB 2.478 13.23 3.4 0.0 ... 0.001 0.0 0.0 11.73

# Fit model for region

In [56]:
region = 'KABETE'
n_knots = 3
degree = 2
polyfeatures = 1

idx_region =  bounding_box(region,meta[region].values,min_x=box_left,max_x=box_right,min_y=box_down,max_y=box_up)

train_data_y_region = train_data_y[region].values
train_data_x_region = train_data_x.isel({'latlon':idx_region}).Nairobi.values

if train_data_x_region.shape[1]==1000:
    train_data_x_region = np.percentile(train_data_x_region,np.linspace(1,100,50),axis=1,method='weibull').T

# Manually make pipeline in future can use from sklearn.pipeline import make_pipeline
spline = SplineTransformer(n_knots=n_knots, degree=degree, knots='quantile').fit(train_data_x_region)
polyfeatures = PolynomialFeatures(polyfeatures, interaction_only=True).fit(spline.transform(train_data_x_region))

transformed_x_region = polyfeatures.transform(spline.transform(train_data_x_region))

logreg = LogisticRegression(penalty='elasticnet',
                            l1_ratio=0.6,solver='saga',max_iter=100_000).fit(transformed_x_region,
                                                                             np.searchsorted([2,20,50],train_data_y_region))


In [57]:
## Define the bin for which we plot the calibraiton curve (i.e., 0, 1, 2, 3 for <5, 5-20, 20-50 and >50 respectively)
bin_to_plot = 2

## We get the true probabilities of occurrence for this bin as is in our y-vales
## we use np.where for this, see documentation here: https://numpy.org/doc/stable/reference/generated/numpy.where.html
actual_probabilities = np.where(train_data_y_region==bin_to_plot,1,0)

## Now we get the predicted probabilities from the fitted models using the function predict_proba and
## The output of predict proba is of the shape (n_samples, n_bins) and we select our bin (i.e., [:,bin_to_plot] of interest
pred_probabilities = logreg.predict_proba(polyfeatures.transform(spline.transform(train_data_x_region)))[:,bin_to_plot]




In [60]:
pred_probabilities

array([0.08782991, 0.12155618, 0.06490838, 0.03446279, 0.00925796,
       0.00586921, 0.00921844, 0.0114451 , 0.02192114, 0.01002407,
       0.00352889, 0.00314557, 0.00276619, 0.00259042, 0.00288908,
       0.0037873 , 0.0043723 , 0.00378503, 0.00324406, 0.00250734,
       0.00259947, 0.00256916, 0.00293566, 0.00693033, 0.01475325,
       0.0025319 , 0.00259427, 0.00267331, 0.00315174, 0.00264819,
       0.00245632, 0.00253   , 0.00278306, 0.00293057, 0.00292457,
       0.00248164, 0.00250452, 0.00221571, 0.00253634, 0.0027269 ,
       0.00261355, 0.00280489, 0.0032527 , 0.00245416, 0.00286931,
       0.00332926, 0.00627899, 0.01013672, 0.00902193, 0.00322765,
       0.00337182, 0.00270623, 0.00261567, 0.00289704, 0.00273987,
       0.00336191, 0.39247864, 0.34218106, 0.30713082, 0.46176824,
       0.28161976, 0.19623414, 0.01927733, 0.02242516, 0.03078537,
       0.0331756 , 0.06237829, 0.27110585, 0.16165479, 0.41036365,
       0.41252474, 0.35631767, 0.24214982, 0.28100761, 0.06034